### Attempt 1: Chignolin, automatic assembly

In [1]:
from pyclops import ChemicalLossHandler
from pyclops.visualization.nglview

In [8]:
chig_pdb = '/Users/bendupontjr/mphil_files/all_mphil_code/pyclops/examples/pdbs/chignolin.pdb'
clh = ChemicalLossHandler.from_pdb_file(chig_pdb, units_factor = 1.0)

print(clh.summary)

AmideAspHead [0, 2] (x2 resonance groups), AmideGluHead [0, 4] (x2 resonance groups), AmideHead2Tail [0, 9] (x2 resonance groups), AspCTerm [2, 9] (x4 resonance groups), AspGlu [2, 4] (x4 resonance groups), GluCTerm [4, 9] (x4 resonance groups)


### Parsing a RDKit molecule

In [1]:
import MDAnalysis as mda
import MDAnalysis.converters.RDKitParser as RDKitParser
import rdkit.Chem as Chem

In [10]:
pdb_file = '/Users/bendupontjr/mphil_files/all_mphil_code/pyclops/examples/pdbs/chignolin.pdb'
mol = Chem.MolFromPDBFile(pdb_file, sanitize=True, removeHs=True)

#u = RDKitParser.RDKitParser(pdb_file)

#mol.GetAtomWithIdx(5).GetPDBResidueInfo().GetResidueName()

mol.GetAtomWithIdx(12).GetPDBResidueInfo().GetName().strip()

'N'

In [43]:
len(mol.GetAtoms())

93